In [34]:
import pandas as pd
import httpx
from bs4 import BeautifulSoup
import asyncio
from datetime import datetime, timedelta
import sqlite3
import nest_asyncio

In [35]:
all_box_urls = {
    'Football': "https://nflbox.me/football-streams",
    'Basketball': "https://www.nbabox.me/watch-baketball-online",
    'Baseball': "https://mlbbox.me/baseball-streams",
    "Hockey": "https://nhlbox.me/hockey-streams",
    "UFC": "https://mmastreams.me/ufc-streams",
    "Racing": "https://f1box.me/motor-racing-streams",
    "Golf": "https://golfstreams.me/live-golf-streams"
}

nest_asyncio.apply()
async def fetch_and_parse(client, box_name, url):
    """Fetch and parse the webpage."""
    response = await client.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    a_tags = soup.find_all('a')
    # Filter tags with "aria-controls" in them
    filtered_a_tags = [str(tag) for tag in a_tags if "aria-controls" in str(tag)]
    return box_name, filtered_a_tags

async def fetch_all_box_a_tags():
    """Main asynchronous function to fetch all URLs."""
    all_box_a_tags = {}
    async with httpx.AsyncClient() as client:
        # Create tasks for fetching all URLs
        tasks = [fetch_and_parse(client, name, url) for name, url in all_box_urls.items()]
        # Wait for all tasks to complete
        results = await asyncio.gather(*tasks)
        # Populate the results into the dictionary
        for box_name, a_tags in results:
            all_box_a_tags[box_name] = a_tags
    return all_box_a_tags

def get_cached_dataframe():
    """Fetch, parse, and cache the DataFrame."""
    all_box_a_tags = asyncio.run(fetch_all_box_a_tags())
    allBox_df = []

    for sport in all_box_a_tags:
        for tag in all_box_a_tags[sport]:
            game_name = tag.split('title="')[1].split('"')[0]
            href = tag.split('href="/')[1].split('"')[0]
            # href = f"{href.split('/')[1]}/{href.split('/')[0]}"
            href = f"{min(href.split('/'), key=len)}/{max(href.split('/'), key=len)}"

            try:
                start_date = tag.split('content="')[1].split('"')[0]
                # Parse the string into a datetime object
                start_date = datetime.strptime(start_date, "%Y-%m-%dT%H:%M")
                # Subtract 6 hours
                start_date = start_date - timedelta(hours=6)
                # Convert back to string if needed
                start_date = start_date.strftime("%Y-%m-%dT%H:%M")
            except Exception as e:
                start_date = None
            allBox_df.append({
                'sport': sport,
                "game_name": game_name,
                'href': href,
                'start_date': start_date
            })

    allBox_df = pd.DataFrame(allBox_df)
    return allBox_df.sort_values(by='start_date', na_position='first')

# Fetch the cached DataFrame
allBox_df = get_cached_dataframe()

def format_start_date(row):
    if row["start_date"] is None:
        return "Live Television"
    return row["start_date"]
def extract_date_and_time_with_label(start_date):
    if start_date is None:
        return None, "Live Television"
    else:
        dt = datetime.fromisoformat(start_date)
        date = dt.strftime("%Y-%m-%d")  # Extract only the date
        time = dt.strftime("%I:%M %p")  # 12-hour format with AM/PM
        
        # Calculate relative day (Today, Tomorrow, etc.)
        today = datetime.now().date()
        game_date = dt.date()
        if game_date == today:
            day_label = "Today"
        elif game_date == today + timedelta(days=1):
            day_label = "Tomorrow"
        else:
            day_label = game_date.strftime("%A")  # Day of the week
        
        return date, f"{time} ({day_label})"

# Apply function to split start_date and add labels
allBox_df[["start_date", "start_time"]] = allBox_df["start_date"].apply(
    lambda x: pd.Series(extract_date_and_time_with_label(x))
)
allBox_df["display_date"] = allBox_df.apply(format_start_date, axis=1)

conn = sqlite3.connect('boxStreams.db')
allBox_df.to_sql("boxStreams", conn, if_exists="replace", index=False)
conn.close()


In [36]:
allBox_df

,sport,game_name,href,start_date,start_time,display_date
0,Football,ESPN,nfl-news/espn-stream,None,Live Television,Live Television
1,Football,NFL Network,nfl/nfl-network-stream,None,Live Television,Live Television
2,Football,NFL RedZone,nfl/nfl-redzone-stream,None,Live Television,Live Television
3,Football,MNF: Monday Night Football,nfl/mnf-monday-night-football-stream,None,Live Television,Live Television
4,Football,TNF: Thursday Night Football,nfl/tnf-thursday-night-football-stream,None,Live Television,Live Television
...,...,...,...,...,...,...
90,Racing,IMSA Rolex 24: Part 4,fia-wtcc/imsa-rolex-24-part-4-stream,2025-01-26,07:00 AM (Sunday),2025-01-26
91,Racing,AMSOIL Championship Snocross Shakopee MN I,auto-racing/amsoil-championship-snocross-shako...,2025-01-26,08:30 AM (Sunday),2025-01-26
92,Racing,AMSOIL Championship Snocross Shakopee MN II,auto-racing/amsoil-championship-snocross-shako...,2025-01-26,09:30 AM (Sunday),2025-01-26
93,Racing,IMSA WeatherTech SportsCar Championship Rolex ...,fia-wtcc/imsa-weathertech-sportscar-championsh...,2025-01-26,11:00 AM (Sunday),2025-01-26


In [37]:
conn = sqlite3.connect('boxStreams.db')
test_df = pd.read_sql_query("""SELECT * FROM boxStreams""", conn)
conn.close()

In [38]:
test_df

,sport,game_name,href,start_date,start_time,display_date
0,Football,ESPN,nfl-news/espn-stream,None,Live Television,Live Television
1,Football,NFL Network,nfl/nfl-network-stream,None,Live Television,Live Television
2,Football,NFL RedZone,nfl/nfl-redzone-stream,None,Live Television,Live Television
3,Football,MNF: Monday Night Football,nfl/mnf-monday-night-football-stream,None,Live Television,Live Television
4,Football,TNF: Thursday Night Football,nfl/tnf-thursday-night-football-stream,None,Live Television,Live Television
...,...,...,...,...,...,...
111,Racing,IMSA Rolex 24: Part 4,fia-wtcc/imsa-rolex-24-part-4-stream,2025-01-26,07:00 AM (Sunday),2025-01-26
112,Racing,AMSOIL Championship Snocross Shakopee MN I,auto-racing/amsoil-championship-snocross-shako...,2025-01-26,08:30 AM (Sunday),2025-01-26
113,Racing,AMSOIL Championship Snocross Shakopee MN II,auto-racing/amsoil-championship-snocross-shako...,2025-01-26,09:30 AM (Sunday),2025-01-26
114,Racing,IMSA WeatherTech SportsCar Championship Rolex ...,fia-wtcc/imsa-weathertech-sportscar-championsh...,2025-01-26,11:00 AM (Sunday),2025-01-26
